### Quick start

This section explains how to get up and running with Nautilus Trader by running some backtests on some Forex data. The Nautilus maintainers have pre-loaded some existing data into the nautilus storage format (parquet) for this guide.

For more details on how to load other data into Nautilus, see #TODO [Backtest Example](../2_user_guide/backtest_example)

#### Getting the sample data

We have prepared some sample data in the nautilus parquet format for use with this example. First, download and unpack the data:

In [6]:
import requests, io, tarfile

#TODO
resp = requests.get("https://raw.githubusercontent.com/limx0/nautilus_data/main/catalogs/EUDUSD202001.tar.gz?token=ABEX2GJNOOJYAPP5AIBWQALB2LLBO")
tarfile.open(fileobj=io.BytesIO(resp.content)).extractall()

In [1]:
from nautilus_trader.persistence.catalog import DataCatalog

In [2]:
catalog = DataCatalog("EUDUSD202001/")

### Writing a trading strategy

Nautilus includes a handful of indicators built-in, in this example we will use a MACD indicator to build a simple trading strategy. You can read more about [MACD here](https://www.investopedia.com/terms/m/macd.asp), but this indicator merely serves as an example without any expected alpha.

In [3]:
from nautilus_trader.trading.strategy import TradingStrategy, TradingStrategyConfig
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model.data.tick import QuoteTick

class MACDConfig(TradingStrategyConfig):
    instrument_id: str


class MACD(TradingStrategy):
    def __init__(self, config):
        super().__init__(config=config)
        self.macd = MovingAverageConvergenceDivergence(fast_period=12, slow_period=26)

    def on_start(self):
        self.subscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        # Update our MACD
        self.macd.handle_quote_tick(tick)



### Configuing Backtests

Now that we have a trading strategy and data, we can run a backtest! Nautilus uses a `BacktestEngine` to configure and run backtests, and requires some setup. This may seem a little complex at first, but this is necessary for the correctness that Nautilus strives for.

To configure a `BacktestEngine`, we create an instance of a `BacktestRunConfig`, configuring the following (minimal) aspects of the backtest:
- `data` - The input data we would like to perform the backtest on
- `venues` - the simulated venues (exchanges or brokers) available in the backtest
- `strategies` - the strategy or strategies we would like to run for the backtest

There are many more configurable features which will be described later in the docs, for now this will get us up and running.

First, we create a venue. For this example we will create a simulated venue for Oanda, a Forex broker. A venue needs a name, as well as some basic configuration; the account type (cash vs margin), the base currency and starting balance.

In [31]:
from nautilus_trader.backtest.config import BacktestVenueConfig
from nautilus_trader.model.enums import VenueType, OMSType, AccountType

oanda_venue = BacktestVenueConfig(
    name="SIM",
    venue_type='BROKERAGE',
    oms_type='NETTING',
    account_type='CASH',
    base_currency="USD",
    starting_balances=['100_000 USD']
)

Next, we need to configure the data for the backtest. Nautilus is built to be very flexible when it comes to loading data for backtests, but this also means configuration is required.

For each

In [34]:
from nautilus_trader.model.data.tick import QuoteTick
from nautilus_trader.backtest.config import BacktestDataConfig

data = [
    BacktestDataConfig(
        catalog_path=str(catalog.path),
        data_cls_path=f"{QuoteTick.__module__}.{QuoteTick.__name__}",
        instrument_id=str(instruments[0].id)
    )
]

Finally, we need a `BacktestEngineConfig` which allows configuring the log level and other components, but is fine to leave with its defaults

In [33]:
from nautilus_trader.backtest.config import BacktestEngineConfig

engine = BacktestEngineConfig()

We can now pass our config to the `BacktestRunConfig`. This object now contains the full configuration for our backtest, we are ready to run some backtests!

In [35]:
config = BacktestRunConfig(
    venues=[oanda_venue],
    data=data,
    engine=engine
)

## Running a backtest

The `BacktestNode` class _actually_ runs the backtest. The reason for this separation between configuration and execution is the `BacktestNode` allows running multiple configurations (different parameters or batches of data), as well as parallelisation via the excellent [dask](https://dask.org/] library.

In [36]:
from nautilus_trader.backtest.node import BacktestNode

node = BacktestNode()
node.run_sync(run_configs=[config]) # run_sync runs one or many configs synchronously